# Sample

In [7]:
import stock 
import plotly.graph_objs as go

import pandas as pd 
import vnstock 

df = stock.get_daily_OHLC(from_date='18/01/2022', to_date='19/01/2024', page_size=1000)
df

,Symbol,Market,TradingDate,Time,Open,High,Low,Close,Volume,Value
0,SSI,HOSE,18/01/2022,None,38504,39199,36635,36766,15992300,699221410000.0020
1,SSI,HOSE,19/01/2022,None,38156,39286,37678,39286,11085500,488587305000.0080
2,SSI,HOSE,20/01/2022,None,39460,40155,38634,39721,9055600,409030700000.0030
3,SSI,HOSE,21/01/2022,None,39286,39634,38417,38721,8527900,383190785000.0010
4,SSI,HOSE,24/01/2022,None,38069,38504,36027,36027,18790500,793994135000.0370
...,...,...,...,...,...,...,...,...,...,...
497,SSI,HOSE,15/01/2024,None,34000,34100,33000,33000,22246200,744181444999.9920
498,SSI,HOSE,16/01/2024,None,33000,33500,32750,33500,19315100,637007325000.0090
499,SSI,HOSE,17/01/2024,None,33500,34500,33450,33650,26218900,892270305000.0280
500,SSI,HOSE,18/01/2024,None,33800,34050,33650,33800,14736700,499010570000.0080


In [8]:
def candlestick_chart(df, title='Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', ma_periods=None, show_volume=True, figure_size=(15, 8), reference_period=None, colors=('#00F4B0', '#FF3747'), reference_colors=('blue', 'black')):
    """
    Generate a candlestick chart with optional Moving Averages (MA) lines, volume data, and reference lines.

    Parameters:
    - df: DataFrame with candlestick data ('time', 'open', 'high', 'low', 'close', 'volume', 'ticker').
    - title: Title of the chart.
    - x_label: Label for the x-axis.
    - y_label: Label for the y-axis.
    - ma_periods: List of MA periods to calculate and plot (e.g., [10, 50, 200]).
    - show_volume: Boolean to indicate whether to display volume data.
    - figure_size: Tuple specifying the figure size (width, height).
    - reference_period: Number of days to consider for reference lines (e.g., 90).
    - colors: Tuple of color codes for up and down candles (e.g., ('#00F4B0', '#FF3747')).
    - reference_colors: Tuple of color codes for reference lines (e.g., ('black', 'blue')).

    Returns:
    - Plotly figure object.
    """
    # Create the base candlestick chart
    candlestick_trace = go.Candlestick(
        x=df['TradingDate'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick',
    )

    # Create a figure
    fig = go.Figure(data=[candlestick_trace])

    # Add volume data if specified
    if show_volume:
        volume_trace = go.Bar(
            x=df['TradingDate'],
            y=df['Volume'],
            name='Volume',
            yaxis='y2',  # Use the secondary y-axis for volume
            marker=dict(color=[colors[0] if close >= open else colors[1] for close, open in zip(df['Close'], df['Open'])]),  # Match volume color to candle color
        )

        fig.add_trace(volume_trace)

    # Add Moving Averages (MA) lines if specified
    if ma_periods:
        for period in ma_periods:
            ma_name = f'{period}-day MA'
            df[ma_name] = df['Close'].rolling(period).mean()

            ma_trace = go.Scatter(
                x=df['TradingDate'],
                y=df[ma_name],
                mode='lines',
                name=ma_name,
            )

            fig.add_trace(ma_trace)

    # Add straight reference lines for the highest high and lowest low
    if reference_period:
        df['lowest_low'] = df['Low'].rolling(reference_period).min()
        df['highest_high'] = df['High'].rolling(reference_period).max()

        lowest_low_trace = go.Scatter(
            x=df['TradingDate'],
            y=[df['lowest_low'].iloc[-1]] * len(df),  # Create a straight line for lowest low
            mode='lines',
            name=f'Lowest Low ({reference_period} days)',
            line=dict(color=reference_colors[0], dash='dot'),
        )

        highest_high_trace = go.Scatter(
            x=df['TradingDate'],
            y=[df['highest_high'].iloc[-1]] * len(df),  # Create a straight line for highest high
            mode='lines',
            name=f'Highest High ({reference_period} days)',
            line=dict(color=reference_colors[1], dash='dot'),
        )

        fig.add_trace(lowest_low_trace)
        fig.add_trace(highest_high_trace)

    # Customize the chart appearance
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        xaxis_rangeslider_visible=True,
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right',
        ),
        width=figure_size[0] * 100,  # Convert short form to a larger size for better readability
        height=figure_size[1] * 100,
        margin=dict(l=50, r=50, t=70, b=50),  # Adjust margins for space between title and legend
    )

    return fig

In [15]:
candlestick_chart(df, ma_periods=[10.])

In [1]:
stock.get_daily_OHLC()

NameError: name 'stock' is not defined